# Cleaning the wrong index

In [1]:
import json


In [6]:
import os

with open("results/ScienceQA/phi3_5/validate.json") as f:
    data = json.load(f)
    n = len(data)
    index = 0
    while n != 0:
        if os.path.isfile(f"datasets/ScienceQA/images/train/{index}.png"):
            data[len(data)-n]['id'] = index
            index += 1
            n -= 1
        else:
            index += 1

In [7]:
print(data[-1])

{'id': 818, 'response': 'Step 1: Identify the information needed to answer the question.\nThe question asks for the capital of Oregon.\n\nStep 2: Compare the options to the information needed.\nOption A is Salem, Option B is Olympia, Option C is Portland, and Option D is Carson City.\n\nStep 3: Determine the correct option based on the information needed.\nThe correct option is (A) Salem.\n\nAnswer: (A)', 'true_false': True}


In [8]:
with open("results/ScienceQA/phi3_5/validate.json", "w") as f:
    json.dump(data, f, indent=4)

# Choose the good responses

In [14]:
with open("results/ScienceQA/phi3_5/validate.json", "r") as f:
    data = json.load(f)
    good_data = []
    true = 0
    false = 0
for i in data:
    if i["true_false"] == False:
        good_data.append(i)
        false += 1
    else:
        if "Step 4" in i["response"]:
            good_data.append(i)
            true += 1
        
print(len(good_data))
print(false)
print(true)
            

125
61
64


In [16]:
with open("results/ScienceQA/phi3_5/validate_good.json", "w") as f:
    json.dump(good_data, f, indent=4)

In [17]:
with open("results/ScienceQA/phi3_5/train.json", "r") as f:
    data = json.load(f)
    good_data = []
    true = 0
    false = 0
for i in data:
    if i["true_false"] == False:
        good_data.append(i)
        false += 1
    else:
        if "Step 4" in i["response"]:
            good_data.append(i)
            true += 1
        
print(len(good_data))
print(false)
print(true)

316
166
150


In [18]:
with open("results/ScienceQA/phi3_5/train_good.json", "w") as f:
    json.dump(good_data, f, indent=4)

In [2]:
with open("results/ScienceQA/phi3_5/train_2.json", "r") as f:
    data = json.load(f)
    good_data = []
    true = 0
    false = 0
for i in data:
    if i["true_false"] == False:
        good_data.append(i)
        false += 1
    else:
        if "Step 4" in i["response"]:
            good_data.append(i)
            true += 1
        
print(len(good_data))
print(false)
print(true)

1426
735
691


In [3]:
with open("results/ScienceQA/phi3_5/train_good.json", "r") as f:
    good_data_1 = json.load(f)
good_data_2 = good_data_1 + good_data
with open("results/ScienceQA/phi3_5/train_2_good.json", "w") as f:
    json.dump(good_data, f, indent=4)

In [1]:
import json
with open("results/ScienceQA/phi3_5/reward.json", "r") as f:
    reward = json.load(f)

In [2]:
len(reward)

1000

# cleaning reward training data

In [1]:
import json

In [2]:
dataset = "ScienceQA"
train_js = f'results/{dataset}/phi3_5/train_2_good.json'
val_js = f'results/{dataset}/phi3_5/train_good.json'
question_js = f'datasets/{dataset}/train.json'
reward_js = f'results/{dataset}/phi3_5/reward_2.json'
reward_js_val = f'results/{dataset}/phi3_5/reward.json'


In [3]:
def read_json(source):
    with open(source, 'r', encoding='utf-8') as f:
        json_list = json.load(f)
    return json_list


train_json = read_json(train_js)
val_json = read_json(val_js)
question_json = read_json(question_js)
reward_json = read_json(reward_js)
reward_json_val = read_json(reward_js_val)

In [15]:
data = []
for idx in range(len(reward_json)):
     # find question and prompt
    i = reward_json[idx]['id']
    placeholder = f"<|image_1|>\n"
    e = (
        "Question: What do these two changes have in common?\ntearing a piece of paper\nbreaking a piece of glass\nContext: N/A\nOptions: (A) Both are only physical changes. (B) Both are chemical changes. (C) Both are caused by cooling. (D) Both are caused by heating.\nStep 1: Identify the trait mentioned in the description.\nThe trait mentioned is \"Logan has blue eyes.\"\n\nStep 2: Determine which option provides information about the origin of the trait.\nOption A states that Logan's mother has blue eyes and passed this trait down to Logan.\n\nStep 3: Compare the information in the options to the trait mentioned.\nOption A provides information about the origin of the trait, while option B does not.\n\nStep 4: Select the option that supports the conclusion that Logan inherited the trait.\n(A) supports the conclusion that Logan inherited the trait.\n\nAnswer: (A)\n\n"
        "Question: What information supports the conclusion that Logan inherited this trait?\nContext: Read the description of a trait.\nLogan has blue eyes.\nOptions: (A) Logan's mother has blue eyes. She passed this trait down to Logan. (B) Logan likes to wear a blue sweater to match his blue eyes.\nStep 1: Identify the type of change for each option.\nTearing a piece of paper is a physical change because it changes the shape and size of the paper, but it does not change the chemical composition of the paper.\nBreaking a piece of glass is also a physical change because it changes the shape and size of the glass, but it does not change the chemical composition of the glass.\n\nStep 2: Compare the types of changes.\nBoth tearing a piece of paper and breaking a piece of glass are physical changes.\n\nStep 3: Select the correct option.\nThe correct option is (A).\n\nAnswer:(A)\n\n"
    )
    query = question_json[i]['input']
    prompt = e + query + "Hint: Please answer the question and provide the correct option letter, e.g., A, B, C, D, at the end. Let's answer step by step.\n"
    # find step and rewards
    s = reward_json[idx].keys()
    s = list(s)[-1]
    p = int(s[-1])
    s_1 = f"Step {p+1}"
    # find corresponding response
    response_list = [d.get("id") for d in train_json]
    j = response_list.index(i)
    response = train_json[j]['response']
    if s_1 in response:
        answer = response.split(s_1)[0]
    elif "Answer" in response:
        answer = response.split("Answer")[0]
    else:
        continue
    
    label = reward_json[idx][s]
    data.append({"id":i, "step_id":p, "input":placeholder + prompt + answer, "label":label})
    

In [16]:
len(data)

2100

In [11]:
with open("results/ScienceQA/phi3_5/reward_train.json", "w") as f:
    json.dump(data, f, indent=4)

In [12]:
data = []
for idx in range(len(reward_json_val)):
     # find question and prompt
    i = reward_json_val[idx]['id']
    placeholder = f"<|image_1|>\n"
    e = (
        "Question: What do these two changes have in common?\ntearing a piece of paper\nbreaking a piece of glass\nContext: N/A\nOptions: (A) Both are only physical changes. (B) Both are chemical changes. (C) Both are caused by cooling. (D) Both are caused by heating.\nStep 1: Identify the trait mentioned in the description.\nThe trait mentioned is \"Logan has blue eyes.\"\n\nStep 2: Determine which option provides information about the origin of the trait.\nOption A states that Logan's mother has blue eyes and passed this trait down to Logan.\n\nStep 3: Compare the information in the options to the trait mentioned.\nOption A provides information about the origin of the trait, while option B does not.\n\nStep 4: Select the option that supports the conclusion that Logan inherited the trait.\n(A) supports the conclusion that Logan inherited the trait.\n\nAnswer: (A)\n\n"
        "Question: What information supports the conclusion that Logan inherited this trait?\nContext: Read the description of a trait.\nLogan has blue eyes.\nOptions: (A) Logan's mother has blue eyes. She passed this trait down to Logan. (B) Logan likes to wear a blue sweater to match his blue eyes.\nStep 1: Identify the type of change for each option.\nTearing a piece of paper is a physical change because it changes the shape and size of the paper, but it does not change the chemical composition of the paper.\nBreaking a piece of glass is also a physical change because it changes the shape and size of the glass, but it does not change the chemical composition of the glass.\n\nStep 2: Compare the types of changes.\nBoth tearing a piece of paper and breaking a piece of glass are physical changes.\n\nStep 3: Select the correct option.\nThe correct option is (A).\n\nAnswer:(A)\n\n"
    )
    query = question_json[i]['input']
    prompt = e + query + "Hint: Please answer the question and provide the correct option letter, e.g., A, B, C, D, at the end. Let's answer step by step.\n"
    # find step and rewards
    s = reward_json_val[idx].keys()
    s = list(s)[-1]
    p = int(s[-1])
    s_1 = f"Step {p+1}"
    # find corresponding response
    response_list = [d.get("id") for d in train_json]
    j = response_list.index(i)
    response = train_json[j]['response']
    if s_1 in response:
        answer = response.split(s_1)[0]
    elif "Answer" in response:
        answer = response.split("Answer")[0]
    else:
        continue
    
    label = reward_json_val[idx][s]
    data.append({"id":i, "step_id":p, "input":placeholder + prompt + answer, "label":label})
    

In [13]:
len(data)

1000

In [14]:
with open("results/ScienceQA/phi3_5/reward_valid.json", "w") as f:
    json.dump(data, f, indent=4)

In [ ]:
import json
with open("results/ScienceQA/phi3_5/deepspeed_config.json", "w") as f:
    data = {
    "fp16": {
        "enabled": true,
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "weight_decay": "auto",
            "torch_adam": true,
            "adam_w_mode": true
        }
    },
    "scheduler": {
        "type": "WarmupDecayLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto",
            "total_num_steps": "auto"
        }
    },
    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": true,
        "allgather_bucket_size": 2e8,
        "overlap_comm": true,
        "reduce_scatter": true,
        "reduce_bucket_size": "auto",
        "contiguous_gradients": true
    },
    "gradient_accumulation_steps": 1,
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}
    json.dump(data, f, indent=4)

# test data clening

In [9]:
import json

dataset = "ScienceQA"
test_js = f'results/{dataset}/phi3_5/test.json'
question_js = f'datasets/{dataset}/test.json'

In [10]:
def read_json(source):
    with open(source, 'r', encoding='utf-8') as f:
        json_list = json.load(f)
    return json_list


test_json = read_json(test_js)
question_json = read_json(question_js)

In [11]:
import os

data = []
for i in test_json:
    question = question_json[i['id']]
    placeholder = f"<|image_1|>\n"
    e = (
        "Question: What do these two changes have in common?\ntearing a piece of paper\nbreaking a piece of glass\nContext: N/A\nOptions: (A) Both are only physical changes. (B) Both are chemical changes. (C) Both are caused by cooling. (D) Both are caused by heating.\nStep 1: Identify the trait mentioned in the description.\nThe trait mentioned is \"Logan has blue eyes.\"\n\nStep 2: Determine which option provides information about the origin of the trait.\nOption A states that Logan's mother has blue eyes and passed this trait down to Logan.\n\nStep 3: Compare the information in the options to the trait mentioned.\nOption A provides information about the origin of the trait, while option B does not.\n\nStep 4: Select the option that supports the conclusion that Logan inherited the trait.\n(A) supports the conclusion that Logan inherited the trait.\n\nAnswer: (A)\n\n"
        "Question: What information supports the conclusion that Logan inherited this trait?\nContext: Read the description of a trait.\nLogan has blue eyes.\nOptions: (A) Logan's mother has blue eyes. She passed this trait down to Logan. (B) Logan likes to wear a blue sweater to match his blue eyes.\nStep 1: Identify the type of change for each option.\nTearing a piece of paper is a physical change because it changes the shape and size of the paper, but it does not change the chemical composition of the paper.\nBreaking a piece of glass is also a physical change because it changes the shape and size of the glass, but it does not change the chemical composition of the glass.\n\nStep 2: Compare the types of changes.\nBoth tearing a piece of paper and breaking a piece of glass are physical changes.\n\nStep 3: Select the correct option.\nThe correct option is (A).\n\nAnswer:(A)\n\n"
    )
    prompts = []
    if os.path.isfile(f"datasets/{dataset}/images/test/{i['id']}.png"):
        text = question['input']
        response = i['response']
        query = e + text + "Hint: Please answer the question and provide the correct option letter, e.g., A, B, C, D, at the end. Let's answer step by step.\n"
        s_id = 1
        s = f"Step {s_id}"
        while s in response:
            s_id += 1
            s = f"Step {s_id}"
            if s in response:
                assistant = response.split(s)[0]
            elif "Answer" in response:
                assistant = response.split("Answer")[0]
            messages = placeholder + query + assistant
            data.append({'id':i['id'], 'Step_id':s_id-1, 'input':messages, "true_false":i["true_false"]})
        

In [12]:
print(len(data))

5713


In [13]:
test_step_js = f'results/{dataset}/phi3_5/test_step.json'
with open(test_step_js, 'w') as f:
    json.dump(data, f)

# Add image path to data

In [ ]:
from utils.json_processor import read_json, write_json
import os
data_path = "datasets/ScienceQA/test.json"
json = read_json(data_path)
for i in range(len(json)):
    
    if os.path.isfile(f"datasets/Science/images/test/{i['id']}.png"):
        
    else:
        image = None
        text = i['input']
        query = e + text + "Hint: Please answer the question and provide the correct option letter, e.g., A, B, C, D, at the end. Let's answer step by step."
        messages = [
            {"role": "user", "content": query},
        ]
    

# Accuracy


In [2]:
def calculate_accuracy(file_path):
    true_count = 0
    total_count = 0
    
    with open(file_path, 'r') as file:
        for line in file:
            # 去掉空白字符并分割
            line = line.strip()
            if line:  # 确保行不为空
                _, value = line.split(':')
                total_count += 1
                if value.strip() == "True":
                    true_count += 1
    
    if total_count == 0:
        print("文件为空或没有有效数据！")
        return
    
    accuracy = (true_count / total_count) * 100
    print(f"总数: {total_count}")
    print(f"True 数量: {true_count}")
    print(f"正确率: {accuracy:.2f}%")

# 将文件路径替换为你的 txt 文件路径
file_path = 'BoN/results/ScienceQA_400.txt'
calculate_accuracy(file_path)


总数: 190
True 数量: 149
正确率: 78.42%
